# Notebook 2: Advanced Chat Capabilities with Oobabooga

## Introduction

Welcome to the advanced course! We're diving into more complex chat capabilities. Hold tight!

## Setup

Just like in Notebook 01, let's set up the environment first.

**Step 1**: Configure your Oobabooga service settings

Use [this notebook](0-AI-settings.ipynb) to save your Oobabooga settings in the configuration file.
Here, we load the settings section relevant to this notebook.

Note that we use the multiconnector nuget package because it provides us with helper classes to deal with Oobabooga configuration in Settings.json, but we won't be using the multiconnector capabilities apart from this.

In [1]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs

//Import package for loading hierarchical settings from settings.json
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.Json"
#r "nuget: Microsoft.Extensions.Configuration.Binder"

//Import multiconnector package for loading hierarchichal settings from settings.json, discarding the previous import

#r "nuget: MyIA.SemanticKernel.Connectors.AI.MultiConnector"

using MyIA.SemanticKernel.Connectors.AI.MultiConnector.Configuration;
using System.IO;

// Load configuration using builder package

using Microsoft.Extensions.Configuration;

var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/settings.json", optional: false, reloadOnChange: true);

IConfiguration configuration = builder.Build();

var oobaboogaConfiguration = configuration.GetSection("Oobabooga").Get<OobaboogaConnectorConfiguration>();

Installed Packages Microsoft.Extensions.Configuration, 7.0.0 Microsoft.Extensions.Configuration.Binder, 7.0.4 Microsoft.Extensions.Configuration.Json, 7.0.0 MyIA.SemanticKernel.Connectors.AI.MultiConnector, 0.34.3

**Step 2**: Import Semantic Kernel SDK and Oobabooga from NuGet

We're importing the big guns again. Semantic Kernel and Oobabooga, come on in!

In [2]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel"
// Import Oobabooga connector
#r "nuget: MyIA.SemanticKernel.Connectors.AI.Oobabooga"

Installed Packages microsoft.semantickernel, 1.0.0-beta6 myia.semantickernel.connectors.ai.oobabooga, 0.34.3


**Step 3**: Create Oobabooga Chat completion settings

We're setting up chat completion parameters. It's got the same parameters as text completion, including generation presets, plus all the oobabooga parameters for chat, including character presets.
We're only dealing with the basic endpoint parameters here.

In [3]:
using MyIA.SemanticKernel.Connectors.AI.Oobabooga.Completion.ChatCompletion;
using System.Text.Json;

//Build settings from configuration file

var oobaboogaChatCompletionSettings = new OobaboogaChatCompletionSettings(
    endpoint: new Uri(oobaboogaConfiguration.EndPoint),
    blockingPort: oobaboogaConfiguration.BlockingPort,
    streamingPort: oobaboogaConfiguration.StreamingPort
);

// Serialize to JSON
string jsonString = JsonSerializer.Serialize(oobaboogaChatCompletionSettings, new JsonSerializerOptions { WriteIndented = true });

// Display the JSON string
Console.WriteLine($"Serialized settings: {jsonString}");


Serialized settings: {
  "OverrideRequestSettings": false,
  "OobaboogaParameters": {
    "mode": "chat",
    "character": "Example",
    "instruction_template": null,
    "your_name": "You",
    "regenerate": false,
    "_continue": false,
    "stop_at_newline": false,
    "chat_generation_attempts": 1,
    "chat-instruct_command": "Continue the chat dialogue below. Write a single reply for the character \u0022\u003C|character|\u003E\u0022.\n\n\u003C|prompt|\u003E",
    "context_instruct": "",
    "max_new_tokens": null,
    "auto_max_new_tokens": null,
    "preset": "None",
    "do_sample": true,
    "temperature": 0,
    "top_p": 0,
    "typical_p": 1,
    "epsilon_cutoff": 0,
    "eta_cutoff": 0,
    "tfs": 1,
    "top_a": 0,
    "repetition_penalty": 1.18,
    "top_k": 20,
    "min_length": 0,
    "no_repeat_ngram_size": 0,
    "num_beams": 1,
    "penalty_alpha": 0,
    "length_penalty": 1,
    "early_stopping": false,
    "mirostat_mode": 0,
    "mirostat_tau": 5,
    "mirostat_

## Basic Chat Completion

Let's start simple. We're adapting a basic chat test to this notebook. You'll ask a question, and Oobabooga will answer. Note that default settings use the Example character presets. Her name is Chiharu Yamada. Check here out in Oobabooga's user interface.

In [4]:
using Microsoft.SemanticKernel.AI.ChatCompletion;
using System.Threading;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);

var history = new ChatHistory();
var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

var localResponse = await oobaboogaLocal.GetChatCompletionsAsync(history, new OobaboogaChatCompletionRequestSettings()
{
    Temperature = 0.01,
    MaxNewTokens = 20,
    TopP = 0.1,
});

var chatMessage = await localResponse[^1].GetChatMessageAsync(CancellationToken.None).ConfigureAwait(false);
Console.WriteLine($"{chatMessage.Role}: {chatMessage.Content}");

user: What is your name?
assistant: Chiharu Yamada.


## Streaming Chat Completion

Now, let's make it a bit more real-time with streaming. Note that responses bits are streamed in a funny way, so you may want to adapt that.

In [5]:
using System.Text;
using Microsoft.SemanticKernel.AI.ChatCompletion;



ChatMessageBase? chatMessage = null;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);
var history = new ChatHistory();

var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

var localResponse = oobaboogaLocal.GetStreamingChatCompletionsAsync(history, new OobaboogaChatCompletionRequestSettings()
{
    Temperature = 0.01,
    MaxNewTokens = 10,
    TopP = 0.1,
});

await foreach (var result in localResponse)
{
    await foreach (var message in result.GetStreamingChatMessageAsync())
    {
        Console.WriteLine($"{message.Role}: {message.Content}");
        chatMessage = message;
    }
}


user: What is your name?
assistant: Ch
assistant: iharu
assistant:  Yamada
assistant: .
assistant: 


To make the output more user-friendly, we can adapt our console writing routine to only write the new characters received. Here's how:
We first define a method in charge of doing the characters processing.

In [6]:
async Task HandleStreamingLoop(IAsyncEnumerable<IChatStreamingResult> localResponse, StringBuilder assistantResponse)
{
    bool isFirstMessage = true;
    await foreach (var result in localResponse)
    {
        await foreach (var message in result.GetStreamingChatMessageAsync())
        {
            if (isFirstMessage)
            {
                Console.Write($"{message.Role}: ");
                isFirstMessage = false;
            }
            Console.Write(message.Content);
        }
    }
}


Then we can use our new method within a conversation:

In [7]:
StringBuilder assistantResponse = new StringBuilder();
ChatMessageBase? chatMessage = null;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);
var history = new ChatHistory();

var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

var localResponse = oobaboogaLocal.GetStreamingChatCompletionsAsync(history, new OobaboogaChatCompletionRequestSettings()
{
    Temperature = 0.01,
    MaxNewTokens = 10,
    TopP = 0.1,
});

await HandleStreamingLoop(localResponse, assistantResponse);


user: What is your name?
assistant: Chiharu Yamada.

## Advanced Chat with Kernel

Let's see how to configure your oobabooga chat service into the kernel, and how to handle an interactive session

In [8]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using MyIA.SemanticKernel.Connectors.AI.Oobabooga;
// using System.Diagnostics;

// Configure the two AI features: OpenAI Chat and DALL-E 2 for image generation
var builder = new KernelBuilder();

builder.WithOobaboogaChatCompletionService(oobaboogaChatCompletionSettings);

// Debugger.Break();
var kernel = builder.Build();

// Get AI service instance used to manage the user chat
var oobaboogaLocal = kernel.GetService<IChatCompletion>();

### Chat configuration

Before starting the chat, we create a new chat object with some instructions, which are included in the chat history. 

The instructions tell OpenAI what kind of chat we want to have

In [9]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;

var systemMessage = "You're chatting with a user. Provide helpful and accurate responses.";

var chat = oobaboogaLocal.CreateNewChat(systemMessage);


### Let's Chat

Run the following code to start the chat. The chat consists of a loop with these main steps:

1. Ask the user for a message. Add the user message into the Chat History object.
2. Send the chat object to Oobabooga asking to generate a response. Add the bot message into the Chat History object.
3. Show the answer to the user.


In [10]:


var userMessage = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Your message (or type 'exit' to exit)");
while (userMessage!="exit")
{
    // 1. Ask the user for a message and add it to the Chat History object.
    
    chat.AddUserMessage(userMessage);
    Console.WriteLine($"user: {userMessage}");

    // 2. Send the chat object to Oobabooga to generate a response.
        var localResponse = await oobaboogaLocal.GetChatCompletionsAsync(chat, new OobaboogaChatCompletionRequestSettings(){
     Temperature = 0.01,
     MaxNewTokens = 100,
     TopP = 0.1,
     });
    var assistantMessage = await localResponse[^1].GetChatMessageAsync(CancellationToken.None).ConfigureAwait(false);
    chat.AddAssistantMessage(assistantMessage.Content);

    // 3. Show the answer to the user.
    Console.WriteLine($"{assistantMessage.Role}: {assistantMessage.Content}");
    userMessage = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Your message (or type 'exit' to exit)");
}


user: Hi, I have a question in astrophysics
assistant: Oh? Well, I don&#x27;t specialize in that field, but I can try my best to help you if it involves computers or programming.
user: What is located at the center of our galaxy?
assistant: The supermassive black hole known as Sagittarius A* (pronounced &quot;Sagittarius A-star&quot; or simply &quot;A-star&quot;) lies at the very center of the Milky Way Galaxy. It has a mass that is about 4 million times that of our Sun and exerts an immense gravitational pull on nearby stars and gas clouds, influencing their orbital paths and behavior.
user: How did astrophysicists figure that out?
assistant: They used mathematical models to predict the behavior of black holes based on Einstein&#x27;s theory of general relativity. Then they compared those predictions with observations from telescopes and other instruments, like gravitational wave detectors. As more data became available, their understanding of black hole dynamics improved, allowing th

## MultiStream Chat Completion with Oobabooga

If you're looking for advanced capabilities, you can simulate MultiStream chat completion using multiple Oobabooga connectors with distinct settings. This example will show you how to set up two connectors and stream chat completions asynchronously from both.

### Initialize Secondary Connector

First, let's get the secondary endpoint and streaming port from the user.

In [11]:
var secondaryEndpoint = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter the secondary endpoint");
var secondaryStreamingPort = int.Parse(await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter the secondary streaming port"));

Now, initialize the secondary Oobabooga connector.

In [12]:
var secondaryOobaboogaSettings = new OobaboogaChatCompletionSettings(
    endpoint: new Uri(secondaryEndpoint),
    streamingPort: secondaryStreamingPort
);

var secondaryChatCompletion = new OobaboogaChatCompletion(secondaryOobaboogaSettings);


Let's now introduce 2 displays, helper methods to update them asynchronously and perform the streaming for each connector.

In [13]:
using System;

// Helper method to update the display in real-time
void UpdateDisplay(string role, string message, Action<string> updateAction)
{
    var output = $"{role}: {message}";
    updateAction(output);
}

// StreamChatCompletionAsync method for handling streaming chat completion
async Task<string> StreamChatCompletionAsync(IChatCompletion chatCompletion, string role, ChatHistory chatHistory, Action<string> updateAction)
{
    var accumulator = new StringBuilder();
    await foreach (var result in chatCompletion.GetStreamingChatCompletionsAsync(chatHistory, new OobaboogaChatCompletionRequestSettings(){
     Temperature = 1.5,
     MaxNewTokens = 100,
     TopP = 0.9,
     }))
    {
        
        await foreach (var message in result.GetStreamingChatMessageAsync())
        {
            accumulator.Append(message.Content);
            UpdateDisplay(role, accumulator.ToString(), updateAction);
        }
    }
    return accumulator.ToString();
}


Now we can stream the 2 chats asynchronously

In [14]:
// Initialize display handles
var primaryDisplay = display("Primary connector is initializing...");
var secondaryDisplay = display("Secondary connector is initializing...");

// Create instances for both connectors
var primaryChatCompletion = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);

// Create chat history and add a user message
var chatHistory = new ChatHistory();
chatHistory.AddUserMessage("What is your name?");

// Run both streaming chat completions asynchronously using StreamChatCompletionAsync
var primaryTask = StreamChatCompletionAsync(primaryChatCompletion, "Oobabooga Primary", chatHistory, content => primaryDisplay.Update(content));
var secondaryTask = StreamChatCompletionAsync(secondaryChatCompletion, "Oobabooga Secondary", chatHistory, content => secondaryDisplay.Update(content));

// Wait for both tasks to complete
await Task.WhenAll(primaryTask, secondaryTask);


Oobabooga Primary: My name&#x27;s Chiharu Yamada. Nice to meet you!

Oobabooga Secondary: Chiharu Yamada.

## Multi-Model Chat with Oobabooga and ChatGPT

In this section, we'll take it up a notch. We'll have ChatGPT talking with two Oobabooga models. The first model to finish responding will continue the conversation with ChatGPT. This will give you a real-time view of how these models interact.

**Pre-requisites**: Make sure you have your OpenAI API credentials configured. If you haven't done this yet, head over to the [settings notebook](0-AI-settings.ipynb) to set it up.

### Step 1: Initialize ChatGPT

First, let's initialize ChatGPT. We'll use the OpenAI API for this.

In [15]:
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;

var openAIConfiguration = configuration.GetSection("OpenAI").Get<OpenAIConfiguration>();

IChatCompletion chatGPT = new OpenAIChatCompletion(
    openAIConfiguration.ChatModelId,
    openAIConfiguration.ApiKey);


### Step 2: Initiating 2 conversations.

We'll deal with 2 chat histories, one for chatgpt and one for the 2 oobabooga models. On each message, the fastest model gets to continue the conversation.

In [16]:
// Initialize chat histories
var chatHistory1 = new ChatHistory();
var chatHistory2 = new ChatHistory();

chatHistory1.AddSystemMessage(@"You are discussing with two large language models smaller than you are.
 The fastest model to answer your message gets to send you the following message. 
 Please try to assess their skills, capabilities, personae. Keep asking them questions and don't let them lead on the conversation.");
chatHistory2.AddSystemMessage("You're chatting with ChatGPT, which should be more intelligent that your are. Or maybe prove us wrong. Let's see");

// Add initial user message to both chat histories
var initialMessage = "Hi ChatGPT, I was told you have a couple questions for me. What is this about?";
chatHistory1.AddUserMessage(initialMessage);
chatHistory2.AddAssistantMessage(initialMessage);

### Step 2: The Main Loop

Now, let's put it all together in a loop that will manage the conversation.

In [17]:
display($"Oobabooga {initialMessage}");
for (int i = 0; i < 12; i++)
{
    // Create new displays for each round
    var chatGPTDisplay = display("ChatGPT is thinking...");
    

    // ChatGPT talks to Oobabooga models
    var chatGPTTask = StreamChatCompletionAsync(chatGPT, "ChatGPT", chatHistory1, content => chatGPTDisplay.Update(content));
    
    
    // Wait for ChatGPT to finish and update chat histories
    await chatGPTTask;
    var chatGPTResponse = chatGPTTask.Result;
    chatHistory1.AddAssistantMessage(chatGPTResponse);
    chatHistory2.AddUserMessage(chatGPTResponse);

    // Oobabooga models talk to ChatGPT

    var primaryDisplay = display("Primary Oobabooga is thinking...");
    var secondaryDisplay = display("Secondary Oobabooga is thinking...");
    
    var primaryTask = StreamChatCompletionAsync(primaryChatCompletion, "Oobabooga Primary", chatHistory2, content => primaryDisplay.Update(content));
    var secondaryTask = StreamChatCompletionAsync(secondaryChatCompletion, "Oobabooga Secondary", chatHistory2, content => secondaryDisplay.Update(content));


    // Wait for the first Oobabooga model to finish
    var firstResponder = await Task.WhenAny(primaryTask, secondaryTask);
    var firstResponse = firstResponder.Result;
    chatHistory1.AddUserMessage(firstResponse);
    chatHistory2.AddAssistantMessage(firstResponse);


}

Oobabooga Hi ChatGPT, I was told you have a couple questions for me. What is this about?

ChatGPT: Hello! Actually, I have a couple of questions for you. I'm here to assess the skills, capabilities, and personae of two language models. I'll be asking them questions and evaluating their responses. Is that okay with you?

Oobabooga Primary: Okay, sure thing! Ask away.

Oobabooga Secondary: Sounds interesting. Sure thing, ask away!

ChatGPT: Great! Let's get started. Here's the first question:

What is the capital of France?

...

Oobabooga Primary: The Pacific Ocean.

Oobabooga Secondary: Oh that&#x27;s easy. The Pacific Ocean is the largest ocean on Earth.

ChatGPT: That's correct! The Pacific Ocean is indeed the largest ocean on Earth. Well done! Now, let's move on to the next question:

Who painted the famous artwork "The Starry Night"?

Oobabooga Primary: Vincent van Gogh painted &quot;The Starry Night

Oobabooga Secondary: Oh wow, great question! It was Vincent van Gogh who created that masterpiece.

Run the above code, and you should see three displays updating in real-time. Each display corresponds to a different model, and you'll see how they interact with each other.

## Conclusion and Next Steps

You've just leveled up your chatbot game! Now, you may dive into the numerous settings provided to control the connector and oobabooga, or learn playing with multiple models thanks to a routing multiconnector.